In [ ]:
import imutils
import cv2
import colorsys
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [20, 8]

In [ ]:
vid = cv2.VideoCapture('/var/tmp/cam1.mp4')
skips = int(vid.get(cv2.CAP_PROP_FPS) / 2)
def make_frames():
    i = 0
    while True:
        status, frame = vid.read()
        if not status:
            break
        i += 1
        if i == skips:
            i = 0
            yield cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        else:
            del frame

frames = list(make_frames())

In [ ]:
len(frames)

In [ ]:
def show(*args):
    if len(args) > 1:
        imgs = np.hstack(tuple(args))
    else:
        imgs = args[0]
    plt.imshow(imgs)
    plt.show()
    
left = frames[39]
right = frames[40]

show(left, right)

In [ ]:
def pp(frame):
    frame = imutils.resize(frame, width = 1000)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    frame = cv2.GaussianBlur(frame, (21, 21), 0)
    return frame
diff = cv2.absdiff(pp(left), pp(right))
show(diff)

In [ ]:
_, thresh = cv2.threshold(diff, 40, 255, cv2.THRESH_BINARY)
show(thresh)

In [ ]:
dilate = cv2.dilate(thresh, None, iterations = 2)
show(dilate)

In [ ]:
onto = left.copy()
scale = left.shape[0]/float(dilate.shape[0])
cnts, _ = cv2.findContours(dilate.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for c in cnts:
    if cv2.contourArea(c) < 500:
        continue
    (x, y, w, h) = cv2.boundingRect(c)
    x = int(x*scale)
    y = int(y*scale)
    w = int(w*scale)
    h = int(h*scale)
    cv2.rectangle(onto, (x,y), (x+w,y+h), (0,255,0), 3)
#onto = cv2.drawContours(onto, cnts, -1, (0,255,0), 3)
show(onto)

In [ ]:
# x, y, w, h
def boxes_for_pair(left, right):
    scaled = cv2.absdiff(pp(left), pp(right))
    scale = left.shape[0]/float(scaled.shape[0])
    _, scaled = cv2.threshold(scaled, 40, 255, cv2.THRESH_BINARY)
    scaled = cv2.dilate(scaled, None, iterations = 2)
    cnts, _ = cv2.findContours(scaled, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for c in cnts:
        if cv2.contourArea(c) < 500:
            continue
        yield [int(p*scale) for p in cv2.boundingRect(c)]

list(boxes_for_pair(left, right))

In [ ]:
def render(left, right):
    onto = left.copy()
    for [x,y,w,h] in boxes_for_pair(left, right):
        cv2.rectangle(onto, (x,y), (x+w,y+h), (0,255,0), 20)
    return onto
show(*[render(frames[x], frames[x+1]) for x in range(0,3)])

In [ ]:
def colour(hue):
    c = colorsys.hsv_to_rgb(hue, 0.8, 0.8)
    return [int(r*256) for r in c]

In [ ]:
onto = frames[0].copy()
for i in range(len(frames) - 1):
    a = frames[i]
    b = frames[i + 1]
    ml = mt = 9001
    mr = mb = 0
    found = False
    for [x,y,w,h] in boxes_for_pair(a, b):
        r = x+w
        b = y+h
        ml = min(x, ml)
        mt = min(y, mt)
        mr = max(r, mr)
        mb = max(b, mb)
        found = True
    if found:
        cv2.rectangle(onto, (ml, mt), (mr, mb), colour(i / float(len(frames))), 20)
show(onto)

In [ ]:
def motions():
    for i in range(len(frames) - 1):
        a = frames[i]
        b = frames[i + 1]
        ml = mt = 9001
        mr = mb = 0
        found = False
        for [x,y,w,h] in boxes_for_pair(a, b):
            r = x+w
            b = y+h
            ml = min(x, ml)
            mt = min(y, mt)
            mr = max(r, mr)
            mb = max(b, mb)
            found = True
        if found:
            yield (i, ml, mt, mr, mb)

In [ ]:
def pluck(img, xy, rb):
    (x, y) = xy
    (r, b) = rb
    return img[y:b, x:r]

show(pluck(frames[0], (350, 1080), (400, 1130)))

In [ ]:
def restack(*imgs):
    h = max(img.shape[1] for img in imgs)
    
    return np.hstack(tuple(imutils.resize(img, height = h) for img in imgs))

show(restack(
    pluck(frames[0], (350, 1080), (400, 1130)),
    pluck(frames[0], (2000, 500), (2200, 700)),
))

In [ ]:
show(restack(*[pluck(frames[i], (x,y),(r,b)) for i,x,y,r,b in list(motions())[1:25]]))

In [ ]:
show(restack(*[pluck(frames[i], (x,y),(r,b)) for i,x,y,r,b in list(motions())[24:]]))

In [ ]:
clear = frames[30]
one = frames[36]
two = frames[37]
show(clear, one, two)

In [ ]:
diff = cv2.absdiff(pp(clear), pp(one))
show(diff)

#_, thresh = cv2.threshold(diff, 40, 255, cv2.THRESH_BINARY)
#show(thresh)

In [ ]:
show(np.uint8((np.float32(clear) + np.float32(one) + np.float32(two))/3))